In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os # 因為要建立資料夾
from tensorflow.keras.models import Sequential # CNN 建立空模函式
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D # CNN 函式
from tensorflow.python.keras.utils import np_utils # 層

In [ ]:
# 掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

# 訓練資料

In [ ]:
import os
path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train'
files = os.listdir(path)
directories = list()

In [ ]:
for dir in files:
  if os.path.isdir(os.path.join(path, dir)):
    directories.append(dir)
directories

In [ ]:
def resizeFile(dir_path, img_files): # dir_path 某個水果資料夾 img_files 資料夾中的所有檔案
  size = (250, 250)
  #size = (150, 150)
  for item in img_files:

    item = os.path.join(dir_path, item)

    images = cv2.imread(item)
    # :2 兩通道
    h, w = images.shape[:2]
    # 找最大邊
    if h > size[0] or w > size[1]:
      # 找出最大比例 => 等比例縮小
      ratio = max(h/size[0], w/size[1])
      images = cv2.resize(images, (int(w/ratio), int(h/ratio)))
      # 重新抓取新的寬、高
    h, w = images.shape[:2]
    # 高度的黑邊
    pad_h = size[0]-h
    # 寬度的黑邊
    pad_w = size[1]-w
    # 上、下補黑邊的高度
    top, bottom = pad_h//2, pad_h-(pad_h//2)
    # 左、右補黑邊的寬度
    left, right = pad_w//2, pad_w-(pad_w//2)
    # 補黑邊(填充：適用於圖片縮小時)
    images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    # 抓檔名
    name = item.split('/')[-1]
    # 用程式判斷是否有所需的資料夾

    cv2.imwrite(f"{dir_path}/resize/{name}", images)

In [ ]:
x_train = list()
y_train = list()

def create_data(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 彩色
    #img_array = cv2.imread(os.path.join(path, p))
    #img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    # 訓練題目
    x_train.append(img_array)
    # 訓練答案
    y_train.append(category)

In [ ]:
category = 0
for directory in directories:
  dir_path = os.path.join(path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeFile(dir_path, img_files)
  # 分類
  create_data(resizeDir, category)
  category += 1


# 測試資料

In [ ]:
import os
test_path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test'
test_files = os.listdir(test_path)
test_directories = list()

In [ ]:
for dir in test_files:
  if os.path.isdir(os.path.join(test_path, dir)):
    test_directories.append(dir)
test_directories

In [ ]:
x_test = list()
y_test = list()

def create_testdata(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 彩色
    #img_array = cv2.imread(os.path.join(path, p))
    #img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    # 訓練題目
    x_test.append(img_array)
    # 訓練答案
    y_test.append(category)

In [ ]:
test_category = 0
for directory in test_directories:
  dir_path = os.path.join(test_path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeFile(dir_path, img_files)
  # 分類
  create_testdata(resizeDir, test_category)
  test_category += 1

# 打散成組的資料

In [ ]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_train = np.array(x_train).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_train = np.array(y_train)

x_train = x_train / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_train = np_utils.to_categorical(y_train)

x_test = np.array(x_test).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_test = np.array(y_test)

x_test = x_test / 255 # 色階 => 收斂
y_test = np_utils.to_categorical(y_test)

# 打亂資料群

In [ ]:
from sklearn.model_selection import train_test_split
# 訓練
rX_train, rX_test, ry_train, ry_test = train_test_split(x_train, y_train, test_size=1, random_state=1)
# 測試
raX_train, raX_test, ray_train, ray_test = train_test_split(x_test, y_test, test_size=1, random_state=1)

# 建模

In [ ]:
model = Sequential()

In [ ]:
# 卷積
model.add(Conv2D(filters=8, kernel_size=(3,3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
# 池化
model.add(MaxPooling2D(pool_size=(2,2))) # pool_size=(2,2) 為預設，不寫也可以
# 卷積
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
model.add(MaxPooling2D(pool_size=(2,2)))
# 卷積
#model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
#model.add(MaxPooling2D(pool_size=(2,2)))
# 放棄的比率
model.add(Dropout(0.3))
# 扁平化
model.add(Flatten())
# 放棄的比率
model.add(Dropout(0.3))
# 連接層：通常用1024(三通道 32*32*32)
model.add(Dense(1024, activation='relu'))
# 輸出層：29種分類
model.add(Dense(29, activation='softmax'))

# 編譯(程式)

In [ ]:
model.compile(optimizer='adam', loss='focal_loss', metrics=['accuracy'])

# 訓練

In [ ]:
model.fit(rX_train, ry_train, epochs=15, batch_size=30)
#, validation_split=0.25

# 測試

In [ ]:
predict1 = model.predict(x_train)

In [ ]:
predict2 = model.predict(raX_train)

In [ ]:
right = 0
for i in range(len(predict1)):
  ans = np.argmax(predict1[i])
  pre = y_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict1)

In [ ]:
right = 0
for i in range(len(predict2)):
  ans = np.argmax(predict2[i])
  pre = ray_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict2)